In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/BankNote_Authentication.xls')
y = df['class']
del df['class']
df

,variance,skewness,curtosis,entropy
0,3.62160,8.66610,-2.8073,-0.44699
1,4.54590,8.16740,-2.4586,-1.46210
2,3.86600,-2.63830,1.9242,0.10645
3,3.45660,9.52280,-4.0112,-3.59440
4,0.32924,-4.45520,4.5718,-0.98880
...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949
1368,-1.38870,-4.87730,6.4774,0.34179
1369,-3.75030,-13.45860,17.5932,-2.77710
1370,-3.56370,-8.38270,12.3930,-1.28230


In [27]:
class Node():

    def __init__(self, value=None):
        self.df = None
        self.parent = None
        self.value = value
        self.left = None
        self.right = None
        self.is_leaf = False

class Tree():

    def __init__(self):
        self.root = None

    def print_tree(self):
        """Печатает дерево в консоли, начиная с корня"""
        if not self.root:
            print("(пустое дерево)")
            return
        
        # Вспомогательная функция с рекурсивным обходом
        def _print(node, prefix="", is_left=True):
            if not node:
                return
            
            # Определяем отображение узла
            if node.is_leaf:
                label = f"{node.value[0]} = {node.value[1]}" if is_left else f"{node.value[0]} = {node.value[1]}"
            else:
                label = f"{node.value[0]} > {node.value[1]}"
            
            print(prefix + ("└── " if not prefix else "├── ") + label)
            
            # Новый префикс для следующих уровней
            new_prefix = prefix + ("    " if not prefix else "│   ")
            
            # Рекурсивно печатаем потомков
            _print(node.left, new_prefix, True)
            _print(node.right, new_prefix, False)
        
        # Запускаем обход от корня
        _print(self.root, "", False)
        
    def find_proba(self, s):
        def tree_traversal(node, s):
            if node.is_leaf:
                return node.value[1]
            else:
                feature, split_val = node.value[0], node.value[1]
                if s[feature] < split_val:
                    return tree_traversal(node.left, s)
                else:
                    return tree_traversal(node.right, s)
        return tree_traversal(self.root, s)
        

class MyTreeClf():
    
    def __init__(self, max_depth=5, min_samples_split=2, max_leafs=20, bins=None, criterion='entropy'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.leafs_cnt = 0
        self.depth = 0
        self.leafs_sum = 0
        self.potential_leafs = 1
        self.tree = Tree()
        self.curr = None
        self.bins = bins
        self.split_bins = None
        self.criterion = criterion
        self.fi = {}

    def __str__(self):
        attrs = vars(self)
        return f'MyTreeClf class: ' + ', '.join(f'{key}={value}' for key, value in attrs.items())

    @staticmethod
    def custom_log(value):
        if value == 0.0:
            return 0
        return np.log2(value)
    
    def entropy_gini(self, N, count_nonzero):
        if self.criterion == 'entropy':
            return -(count_nonzero * MyTreeClf.custom_log(count_nonzero / N) +
                                (N - count_nonzero) * MyTreeClf.custom_log((N - count_nonzero) / N)) / N
        return 1 - (count_nonzero / N) ** 2 - ((N - count_nonzero) / N) ** 2
    
    def get_best_split(self, X, y):
        names = X.columns.values.tolist()
        N = len(y)
        count_nonzero = np.count_nonzero(y)
        d = pd.DataFrame(columns=['col_name', 'split_value', 'ig'])
        S0 = self.entropy_gini(N, count_nonzero)
        X_numpy = X.to_numpy()
        if not self.split_bins:
            for i in range(len(X.columns)):
                feature = pd.DataFrame({i: X_numpy[:,i], 'y': y})
                unique = feature[i].unique()
                unique = np.sort(unique)
                feature_unique_mean = (unique[:-1] + unique[1:]) / 2
                for k in range(len(feature_unique_mean)):
                    left_df = feature.loc[feature[i] <= feature_unique_mean[k]]
                    right_df = feature.loc[feature[i] > feature_unique_mean[k]]
                    left = left_df['y']
                    right = right_df['y']
                    N1 = len(left)
                    N2 = len(right)
                    c_n_l = int(np.count_nonzero(left))
                    c_n_r = int(np.count_nonzero(right))
                    S1 = self.entropy_gini(N1, c_n_l)
                    S2 = self.entropy_gini(N2, c_n_r)
                    ig = S0 - (N1 / N) * S1 - (N2 / N) * S2
                    d.loc[len(d.index)] = [names[i], feature_unique_mean[k], ig]
        else:
            for name in names:
                min_feature, max_feature = min(X[name]), max(X[name])
                splits = self.split_bins[name]
                needed_splits = splits[(splits >= min_feature) & (splits < max_feature)]
                for k in needed_splits:
                    left_df = X.loc[X[name] <= k]
                    right_df = X.loc[X[name] > k]
                    left = self.y[left_df.index]
                    right = self.y[right_df.index]
                    N1 = len(left)
                    N2 = len(right)
                    c_n_l = int(np.count_nonzero(left))
                    c_n_r = int(np.count_nonzero(right))
                    S1 = self.entropy_gini(N1, c_n_l)
                    S2 = self.entropy_gini(N2, c_n_r)
                    ig = S0 - (N1 / N) * S1 - (N2 / N) * S2
                    d.loc[len(d.index)] = [name, k, ig]
        d = d.sort_values(by=['ig', 'col_name'], ascending=[False, True])
        if len(d):
            col_name, split_value, ig = d.iloc[0, 0], d.iloc[0, 1], d.iloc[0, 2]
            return col_name, split_value
        return None, None
    
    def get_split_values_or_bins(self, X):
        if self.bins:
            self.split_bins = {name: None for name in X.columns.values}
            for col in self.split_bins:
                feature = X[col]
                unique = np.sort(feature.unique())
                split_mean = (unique[:-1] + unique[1:]) / 2
                split_hist = np.histogram(feature, bins=self.bins)[1][1:-1]
                if len(split_mean) <= len(split_hist) - 1:
                    self.split_bins[col] = split_mean
                else:
                    self.split_bins[col] = split_hist

    def build_tree(self, X):
        # self.tree.print_tree()
        # print('')
        if self.leafs_cnt >= self.max_leafs or self.depth > self.max_depth:
            return
        if not self.tree.root:
            y = self.y[X.index]
            col_name, split_value = self.get_best_split(X, y)
            self.tree.root = Node((col_name, split_value))
            self.tree.root.df = X
            self.curr = self.tree.root
            X_left = X.loc[X[col_name] <= split_value]
            X_right = X.loc[X[col_name] > split_value]
            self.depth += 1
            self.potential_leafs += 1
            self.fi[col_name] += self.compute_feature_importance(X, X_left, X_right)
            self.build_tree(X_left)
        if not self.curr.left:
            y = self.y[X.index]
            col_name, split_value = self.get_best_split(X, y)
            if not self.is_leaf(y) and col_name: # узел
                self.curr.left = Node((col_name, split_value))
                self.curr.left.parent = self.curr
                self.curr.left.df = X
                self.curr = self.curr.left
                X_left = X.loc[X[col_name] <= split_value]
                X_right = X.loc[X[col_name] > split_value]
                self.fi[col_name] += self.compute_feature_importance(X, X_left, X_right)
                self.depth += 1
                self.potential_leafs += 1
                self.build_tree(X_left)
            else: #  лист
                # print('Зашел в левый лист')
                self.curr.left = Node(('leaf_left', np.count_nonzero(y == 1) / len(y)))
                self.leafs_sum += self.curr.left.value[1]
                self.curr.left.is_leaf = True
                self.curr.left.parent = self.curr
                self.leafs_cnt += 1
                self.potential_leafs -= 1
                self.build_tree(self.curr.df)
        elif self.curr.left and not self.curr.right:
            X_curr = self.curr.df.loc[self.curr.df[self.curr.value[0]] > self.curr.value[1]]
            y_r = self.y[X_curr.index]
            col_name, split_value = self.get_best_split(X_curr, y_r)
            if not self.is_leaf(y_r) and col_name:
                self.curr.right = Node((col_name, split_value))
                self.curr.right.df = X_curr
                self.curr.right.parent = self.curr
                self.curr = self.curr.right
                self.depth += 1
                self.potential_leafs += 1
                X_left = X_curr.loc[X_curr[col_name] <= split_value]
                X_right = X_curr.loc[X_curr[col_name] > split_value]
                self.fi[col_name] += self.compute_feature_importance(X_curr, X_left, X_right)
                self.build_tree(X_left)
            else:
                self.curr.right = Node(('leaf_right', np.count_nonzero(y_r == 1) / len(y_r)))
                self.leafs_sum += self.curr.right.value[1]
                self.curr.right.is_leaf = True
                self.curr.right.parent = self.curr
                if self.tree.root != self.curr:
                    self.curr = self.curr.parent
                self.depth -= 1
                self.leafs_cnt += 1
                self.potential_leafs -= 1
                self.build_tree(self.curr.df)
        elif self.curr == self.tree.root and self.curr.left and self.curr.right:
            return
        elif self.curr.left and self.curr.right:
            self.curr = self.curr.parent
            self.depth -= 1
            self.build_tree(self.curr.df)

    def fit(self, X, y):
        self.y = y
        self.N = len(X)
        self.get_split_values_or_bins(X)
        self.fi.update({col: 0 for col in X.columns.values})
        self.build_tree(X)

    def is_leaf(self, y):
        '''True - лист, False - узел'''
        if len(y) == 1 or np.count_nonzero(y == 1) == 0 or np.count_nonzero(y == 0) == 0 or \
                self.max_depth == self.depth or self.max_leafs <= 2 or self.min_samples_split > len(y) or \
                self.leafs_cnt + self.potential_leafs >= self.max_leafs:
            return True
        return False

    def predict_proba(self, X):
        proba = []
        for ind in X.index.values:
            s = X.loc[ind]
            proba.append(self.tree.find_proba(s))
        return proba
    
    def predict(self, X):
        proba = []
        for ind in X.index.values:
            s = X.loc[ind]
            proba.append(self.tree.find_proba(s))
        pred = [1 if val > 0.5 else 0 for val in proba]
        return pred
    
    def feature_importance(self, N_p, N, N_l, N_r, I, I_l, I_r):
        return N_p * (I - N_l * I_l / N_p - N_r * I_r / N_p) / N
    
    def compute_feature_importance(self, X, X_left, X_right):
        left = self.y[X_left.index]
        right = self.y[X_right.index]
        y = self.y[X.index]
        N = len(X)
        N_l = len(left)
        N_r = len(right)
        c_n_l = int(np.count_nonzero(left))
        c_n_r = int(np.count_nonzero(right))
        c_n = int(np.count_nonzero(y))
        S = self.entropy_gini(N, c_n)
        S1 = self.entropy_gini(N_l, c_n_l)
        S2 = self.entropy_gini(N_r, c_n_r)
        return self.feature_importance(N, self.N, N_l, N_r, S, S1, S2)

In [28]:
lst = [(1,1,2,8, 'gini'), (3,2,5,None,'gini'), (5,200,10,4,'entropy'), (4,100,17,16,'gini'), (10,40,21,10,'gini'), (15,20,30,6,'gini')]
for max_depth, min_samples_split, max_leafs, bins, criterion in lst:
    model = MyTreeClf(max_depth=max_depth, min_samples_split=min_samples_split, max_leafs=max_leafs, bins=bins, criterion=criterion)
    model.fit(df, y)
    print(round(model.leafs_sum, 6))
    print(model.leafs_cnt)
    print(model.fi)

0.981148
2
{'variance': 0.2307416940473953, 'skewness': 0, 'curtosis': 0, 'entropy': 0}
2.799994
5
{'variance': 0.27625318056872344, 'skewness': 0.0702064286269665, 'curtosis': 0, 'entropy': 0}
5.020575
10
{'variance': np.float64(0.42755468161889587), 'skewness': np.float64(0.17088402599288008), 'curtosis': np.float64(0.1283848410531745), 'entropy': 0}
5.200813
11
{'variance': 0.29311785625410264, 'skewness': 0.06980656391496583, 'curtosis': 0.043827837530043955, 'entropy': 0}
10.198869
21
{'variance': 0.28585271146240177, 'skewness': 0.10466537343818459, 'curtosis': 0.05381724716854877, 'entropy': 0.0038454942416527153}
12.412269
27
{'variance': 0.29104459429524376, 'skewness': 0.10539455280100717, 'curtosis': 0.058254028287495305, 'entropy': 0.008363882866077315}


In [24]:
a = {}
a.update({i: 0 for i in range(8)})
a[0] += 3
a[0] += 2
a

{0: 5, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}

In [77]:
# model.split_bins
model.tree.print_tree()

└── variance > -0.10864999999999991
    ├── skewness > 7.606659999999998
    │   ├── curtosis > 6.320649999999998
    │   │   ├── skewness > 4.934189999999997
    │   │   │   ├── curtosis > 1.6779499999999992
    │   │   │   │   ├── leaf_left = 1.0
    │   │   │   │   ├── skewness > 2.261719999999997
    │   │   │   │   │   ├── entropy > -0.8498099999999997
    │   │   │   │   │   │   ├── leaf_left = 0.5
    │   │   │   │   │   │   ├── skewness > -0.41075000000000017
    │   │   │   │   │   │   │   ├── leaf_left = 1.0
    │   │   │   │   │   │   │   ├── leaf_right = 0.9166666666666666
    │   │   │   │   │   ├── leaf_right = 0.0
    │   │   │   ├── leaf_right = 0.8461538461538461
    │   │   ├── skewness > -3.0832200000000007
    │   │   │   ├── variance > -1.4953399999999997
    │   │   │   │   ├── leaf_left = 1.0
    │   │   │   │   ├── leaf_right = 0.7777777777777778
    │   │   │   ├── leaf_right = 0.0
    │   ├── variance > -4.26872
    │   │   ├── leaf_left = 0.9523809523809523
 